# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [19]:
def date_sorter():
    import pandas as pd
    import re
    import numpy as np

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)  
    
    p1=r"(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})"
    p2=r"((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]{2,}.\d{1,2}.\d{4})"
    p3=r"(\d{1,2}.(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.\d{4})"
    p4=r"((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(.{1,3})?(\d{1,}(\w{2})?)?(.{1,3})?\d{4})"
    p5=r"((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]{2,}.\d{4})"
    p6 = r"(\d{1,2}/\d{4})"
    p7=r"(\d{4})"
    p8=r"(\d{1,2}(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.{1,2}?\d{4})"
    p = p1+"|"+p2+"|"+p3+"|"+p4+"|"+p5+"|"+p6+"|"+p7+"|"+p8
   
    pattern = re.compile(p)
    df_dates = df.str.extract(pattern)
    df_dates = df_dates.fillna("9999")

    lis = []
    for item in df_dates.iterrows():
        column_str = ""
        for i in item[1]:
            if i!="9999":
                column_str+=(str(i))
        lis.append(column_str)


    df_dates = pd.Series(lis, name = "date") 
    
    
    
    df_long2 = df_dates[df_dates.str.len()>10]

    def clean_date(date):

        dic = {"Jan":"01", "Feb":"02", "Mar":"03" ,"Apr":"04" ,"May":"05" ,"Jun":"06" ,"Jul":"07" ,"Aug":"08"
               ,"Sep":"09" ,"Oct":"10" ,"Nov":"11" ,"Dec":"12"}

        month = re.findall(r"[A-Za-z]+", date)

        if month:
            month = month[0][:3]

        else:
            month = "01"
        day = re.findall(r"\d{1,2}\b", date)
        if day:
            day = day[0]
        else:
            day = "01"
        year = re.findall(r"\d{4}", date)[0][-2:]
        month = dic[month]
        date = month+"/"+day+"/"+year
        return date
    df_long = df_long2.apply(clean_date)
    df_long = df_long.to_frame()


    df_short1 = df_dates[df_dates.str.len()<6]
    df_short = df_short1.apply(lambda x: "01/01/"+x[-2:])
    df_short = df_short.to_frame()
    # df_short[df_short.str.len()<8]
    
    
    def strip_year(date):
        year = re.findall(r"\d{4}$", date)
        if year:
            year = year[0]
            return date[:-4]+year[-2:]
        else:  
            return date
    
    def add_day(date):
        if len(date)<=5:
            date1 = date.split("/")
            date2 = date1[0]+r"/01/"+date1[1]
            return date2
        else: return date


    df_good = df_dates[(df_dates.str.len()>=6)&(df_dates.str.len()<=10)]
    df_good = df_good.to_frame()
    df_good = df_good.date.apply(strip_year)
    df_good = df_good.apply(add_day)
    df_good[df_good.str.len()<7]
    df_good = df_good.to_frame()

    
    df_all_good1 = pd.merge(df_good, df_short, how="outer", left_index=True, right_index=True, on="date" )
    df_all_good = pd.merge(df_all_good1, df_long, how="outer", left_index=True, right_index=True, on="date" )
    df_all_good.head()
    df_all_good = df_all_good["date"].apply(lambda x: x.strip())
    df_all_good = df_all_good.str.replace(r"[^\d]+", "/")
    df_all_good = df_all_good.to_frame()
    df_all_good = pd.to_datetime(df_all_good["date"], format = "%m/%d/%y")
    df_all_good = df_all_good.to_frame()
    df_all_good.reset_index(inplace=True)
    
    
    
    df_all_good.sort_values(by="date", inplace=True)
    df_all_good.reset_index(inplace=True)
    df_all_good.head()
    output = pd.Series(df_all_good["index"])
    
    return output